In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from traitlets import validate
import sklearn.model_selection
import numpy as np
import tensorflow as tf
import sklearn.metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model


data_dir = r"C:\Users\B.BASI REDDY\Documents\TB_Chest_Radiography_Database\Normal"
data_dir2 = r"C:\Users\B.BASI REDDY\Documents\TB_Chest_Radiography_Database\Tuberculosis"

# Load image filenames
img2 = os.listdir(data_dir2)
images = os.listdir(data_dir)

# Create DataFrame
data = pd.DataFrame({
    'image_path': [os.path.join(data_dir, filename) for filename in images] + [os.path.join(data_dir2, filename) for filename in img2],
    'class': ["NORMAL"] * len(images) + ["TUBERCULOSIS"] * len(img2)
})
print("Number of samples in the dataset:", len(data))


fig, axs = plt.subplots(4, 4, figsize=(10, 10))
for idx, ax in enumerate(axs.ravel()):
    img = cv2.imread(data.iloc[idx]['image_path'], cv2.IMREAD_GRAYSCALE)
    ax.imshow(img, cmap='gray')
    ax.axis('off')
plt.show()


data = data.sample(frac=1).reset_index(drop=True)

# Split the data into training and validation sets
train_split = 0.8
train_size = int(train_split * len(data))

train_data = data.iloc[:train_size]
validation_data = data.iloc[train_size:]

# Display some information about the data
print("Number of samples in the dataset:", len(data))
print("Number of samples in the training set:", len(train_data))
print("Number of samples in the validation set:", len(validation_data))

# ...

# Set up data generators
batch_size = 32
image_size = (128, 128)

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2
)

train_generator = train_datagen.flow_from_dataframe(
    train_data,
    x_col='image_path',
    y_col='class',
    target_size=image_size,
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='binary'
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

validation_generator = validation_datagen.flow_from_dataframe(
    validation_data,
    x_col='image_path',
    y_col='class',
    target_size=image_size,
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='binary'
)

print("Number of samples in the validation set:", len(validation_generator.filenames))
print("Sample validation file paths:", validation_generator.filenames[:5])

sample_img_path = validation_generator.filepaths[0]
sample_img = cv2.imread(sample_img_path, cv2.IMREAD_GRAYSCALE)
plt.imshow(sample_img, cmap='gray')
plt.show()

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    MaxPool2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPool2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator
)

model.save("tbkanukko.keras")

from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model("tbdetection.keras")

# Evaluate the model on the validation data
accuracy = loaded_model.evaluate(validation_generator)[1]
print(f"Accuracy of the loaded model on validation data: {accuracy}")


plt.plot(history.history['accuracy'], label='accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend(loc='lower right')
plt.show()



